# Name List Checker

#### Imports

In [1]:
import os
import ast
import json
import shutil
import platform
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_col

tqdm.pandas()
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
RAW = 'Data/Raw'
IMAGES = 'Output/Images'
TABLES = 'Output/Tables'

In [2]:
doc_list = [
f'{PROCESSED}/monumental_brasses_final.csv',
f'{PROCESSED}/victuallers_list_1651_1828_final.csv',
f'{PROCESSED}/freeholders_list_1713_1780_final.csv',
f'{PROCESSED}/ukda_pcc_wills_final.csv',
f'{PROCESSED}/bank_returns_1845_1880_final.csv',
f'{PROCESSED}/bankrupts_list_1800_1820_final.csv',
f'{PROCESSED}/bankrupts_list_1820_1843_final.csv',
f'{PROCESSED}/indictable_offenses_1745_1782_final.csv',
f'{PROCESSED}/workhouse_list_1861_final.csv',
    ]
desc_list = [
    'Monumental Brasses',
    'Victuallers 1651-1828',
    'Freeholders List 1713-1780',
    'PCC Wills 1558-1858',
    'Bank Returns 1845-1880',
    'Bankrupts List 1800-1820',
    'Bankrupts List 1820-1843',
    'Indictments 1745-1782',
    'Workhouse List 1861',
]
results_df = pd.DataFrame(columns=['Indicator', 'Recipient', 'Control', 'Ratio', 'P-Value'])
for doc_file, description in zip(doc_list, desc_list):
    df = pd.read_csv(doc_file, encoding='utf-8')
    if 'ukda_pcc_wills_final' in doc_file:
        df = df.loc[df['year'] >= 1558]
    indicator = description
    recipient_share = len(df[df['combined_treatment'] == 1]) / len(df)
    control_share = len(df[df['combined_control'] == 1]) / len(df)
    ratio = recipient_share / control_share
    t_test = sm.stats.ttest_ind(df['unique_treatment'], df['unique_control'])
    p_value = t_test[1]
    results_df = pd.concat([results_df, pd.DataFrame({'Indicator': indicator, 'Recipient': recipient_share, 'Control': control_share, 'Ratio': ratio, 'P-Value': p_value}, index=[0])])

# Export to Latex, with some formatting
for col in ['Recipient', 'Control']:
    results_df[col] = results_df[col].apply(lambda x: f'{x:.2%}')
results_df['Ratio'] = results_df['Ratio'].apply(lambda x: f'{x:.2f}')
results_df['P-Value'] = results_df['P-Value'].apply(lambda x: f'{x:.3f}')
# Add some stars
results_df['P-Value'] = results_df['P-Value'].apply(lambda x: f'{x}***' if float(x) < .001 else f'{x}**' if float(x) < .01 else f'{x}*' if float(x) < .05 else x)

latex_text = results_df.to_latex(index=False)
latex_text = latex_text.replace('%', '\\%')
latex_text = '\n\\begin{table}[H]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n' + latex_text + '\n}\n\\end{table}%\n'

with open(f'{TABLES}/status_indicators.tex', 'w', encoding='utf-8') as f:
    f.write(latex_text)

print(results_df)

C:\Users\nicho\AppData\Local\Temp\ipykernel_13232\2113267033.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({'Indicator': indicator, 'Recipient': recipient_share, 'Control': control_share, 'Ratio': ratio, 'P-Value': p_value}, index=[0])])


                    Indicator Recipient Control Ratio   P-Value
0          Monumental Brasses     1.77%   1.33%  1.33    0.013*
0       Victuallers 1651-1828     5.23%   4.39%  1.19  0.000***
0  Freeholders List 1713-1780     5.39%   3.08%  1.75  0.000***
0         PCC Wills 1558-1858     5.07%   3.04%  1.67  0.000***
0      Bank Returns 1845-1880     3.75%   3.79%  0.99   0.009**
0    Bankrupts List 1800-1820     3.82%   5.00%  0.76     0.806
0    Bankrupts List 1820-1843     3.85%   4.62%  0.83     0.463
0       Indictments 1745-1782     4.52%   4.32%  1.04     0.492
0         Workhouse List 1861     2.48%   3.47%  0.71     0.315
